In [ ]:

from query_reader import QueryReader 
import pandas as pd
import psycopg2.extras
import psycopg2
import time
import os

query_dir = os.path.join(os.path.abspath(''), 'queries')
qr = QueryReader(query_dir)
BASE_DATE = "'2021-05-01'"

def create_connection(): 
    return psycopg2.connect(
        dbname='main',
        host='main-us-e2.cmbsiiqeauby.us-east-2.rds.amazonaws.com',
        port=5432,
        user='doadmin',
        password='i39kew8n7jcat7l9'
    )

def run_query(connection, query, cursor=None, stats=False):
    if cursor:
        cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    else:
        cursor = connection.cursor()
    try:
        if stats:
            start_time = time.time()
        cursor.execute(query)
        results = cursor.fetchall()

        if stats:
            end_time = time.time()

        if len(results[0]) == 1:
            results = [x[0] for x in results]
        else:
            results = [x for x in results]
        if stats:
            print(f'Time elapsed: {end_time - start_time}')
            print(f'Length of Results: {len(results)}')
            print(f'Sample Result: {results[0]}')
        return results
    except:
        print('rolling back')
        connection.rollback()
    finally:
        cursor.close()

def big_fetch(connection, query, cursor=None, fetch_size=30000):
    if cursor:
        cursor = connection.cursor('big_fetch_cursos', cursor_factory = cursor)
    else:
        cursor = connection.cursor('big_fetch_crusor')
    results = []
    start_time = time.time()
    cursor.execute(query)
    while True:
        rows = cursor.fetchmany(fetch_size)
        if not rows:
            break
        results.extend(rows)

    end_time = time.time()
    print(f'Time elapsed: {end_time - start_time}')
    print(f'Length of Results: {len(results)}')
    print(f'Sample Result: {results[0]}')
    cursor.close()
    return results


def read_into_df(connection, query, stats=False) -> pd.DataFrame:
    if stats:
        start_time = time.time()
    df = pd.read_sql(query, connection)
    if stats:
        end_time = time.time()
        print(f'Time elapsed: {end_time - start_time}')
        print(f'DF Shape: {df.shape}')
    return df

import warnings
warnings.filterwarnings(action='once')

conn = create_connection()

In [ ]:
kills_query = qr.read_query('csgo_kills.sql')
kills_query = kills_query.format(date=BASE_DATE)

assists_query = qr.read_query('csgo_assists.sql') # 44, 35, 35
assists_query = assists_query.format(date=BASE_DATE)

deaths_query = qr.read_query('csgo_deaths.sql')
deaths_query = deaths_query.format(date=BASE_DATE)

kills_df = read_into_df(conn, kills_query)
assists_df = read_into_df(conn, assists_query)
deaths_df = read_into_df(conn, deaths_query)

In [ ]:
from econ_rebuilder import run_econ_rebuild

econ_query = qr.read_query('econ_rebuild.sql')
econ_query = econ_query.format(date=BASE_DATE)
econ_data = run_query(conn, econ_query, cursor=1, stats=True)
econ_data = [dict(x) for x in econ_data]
econ_df = run_econ_rebuild(econ_data)

In [ ]:
from functools import reduce

econ_cols = [x for x in econ_df.columns if x in ['match_id', 'steam_id'] or x.endswith('_bonus') or x.startswith('bomb_')]
m_econ_df = econ_df[econ_cols]

data_frames = [kills_df, assists_df, deaths_df, m_econ_df]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['match_id', 'steam_id'],
                                            how='outer'), data_frames)

print(df_merged.shape)
df_merged.head()

In [ ]:
print([x for x in df_merged.columns])

In [ ]:
econ_cols = [x for x in econ_df.columns if x in ['match_id', 'steam_id'] or x.endswith('_bonus') or x.startswith('bomb_')]